In [1]:
# Installing kaggle to download the dataset.
!pip install kaggle

In [2]:
# We need kaggle credentials which needs to be stored in ~/.kaggle/kaggle.json
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

In [3]:
import os
from getpass import getpass

# Reading in the contents of user's kaggle.json.
kaggle_secret = getpass('Enter the content of your kaggle.json: ')

# Saving into the previously created file.
with open('/root/.kaggle/kaggle.json', "w") as f:
  f.write(kaggle_secret)

Enter the content of your kaggle.json: ··········


In [4]:
# Downloading and unzipping the training dataset.
!kaggle competitions download -c tpu-getting-started
!unzip tpu-getting-started.zip -d data
!rm tpu-getting-started.zip

100% 4.79G/4.79G [02:41<00:00, 37.2MB/s]
100% 4.79G/4.79G [02:41<00:00, 31.8MB/s]
Archive:  tpu-getting-started.zip
  inflating: data/sample_submission.csv  
  inflating: data/tfrecords-jpeg-192x192/test/00-192x192-462.tfrec  
  inflating: data/tfrecords-jpeg-192x192/test/01-192x192-462.tfrec  
  inflating: data/tfrecords-jpeg-192x192/test/02-192x192-462.tfrec  
  inflating: data/tfrecords-jpeg-192x192/test/03-192x192-462.tfrec  
  inflating: data/tfrecords-jpeg-192x192/test/04-192x192-462.tfrec  
  inflating: data/tfrecords-jpeg-192x192/test/05-192x192-462.tfrec  
  inflating: data/tfrecords-jpeg-192x192/test/06-192x192-462.tfrec  
  inflating: data/tfrecords-jpeg-192x192/test/07-192x192-462.tfrec  
  inflating: data/tfrecords-jpeg-192x192/test/08-192x192-462.tfrec  
  inflating: data/tfrecords-jpeg-192x192/test/09-192x192-462.tfrec  
  inflating: data/tfrecords-jpeg-192x192/test/10-192x192-462.tfrec  
  inflating: data/tfrecords-jpeg-192x192/test/11-192x192-462.tfrec  
  inflating: d

In [5]:
import tensorflow as tf
import numpy as np
import re

# Paths to the training, validation and test datasets.
DATASET_PATH = "data"
PATH = DATASET_PATH + "/tfrecords-jpeg-224x224"
TRAINING_FILENAMES = tf.io.gfile.glob(PATH + "/train/*.tfrec")
VALIDATION_FILENAMES = tf.io.gfile.glob(PATH + "/val/*.tfrec")
TEST_FILENAMES = tf.io.gfile.glob(PATH + "/test/*.tfrec")

# Training related constants.
IMAGE_SIZE = [224, 224]
BATCH_SIZE = 32
NUM_OF_CLASSES = 104

In [6]:
# This function converts the raw image data to a [IMAGE_SIZE, 3] shaped array
# containing the normalized color intensity values for all channels.
def decode_image(image_data):
  # Extracting the image from the dataset.
  image = tf.image.decode_jpeg(image_data, channels=3)
  # Normalizing the color intensity values.
  image = (
      tf.cast(image, tf.float32) / 255.0
  )
  # Reshaping for 3 channels.
  image = tf.reshape(image, [*IMAGE_SIZE, 3])
  return image


# This function takes a raw labelled tfrecord and converts it to decoded image data
# and one hot encoded label.
def read_labelled_tfrecord(example):
  LABELLED_TFREC_FORMAT = {
    "image": tf.io.FixedLenFeature([], tf.string),
    "class": tf.io.FixedLenFeature([], tf.int64),
  }
  # Converting the raw data to a python dictionary with LABELLED_TFREC_FORMAT.
  example = tf.io.parse_single_example(example, LABELLED_TFREC_FORMAT)
  # Decoding the image.
  image = decode_image(example["image"])
  # One-hot encode the label of the image.
  label = tf.cast(example["class"], tf.int32)
  one_hot = tf.one_hot(label, NUM_OF_CLASSES)
  return image, one_hot


# This function takes a raw unlabelled tfrecord and converts it to decoded image data
# and the id of the test image.
def read_unlabelled_tfrecord(example):
  UNLABELLED_TFREC_FORMAT = {
    "image": tf.io.FixedLenFeature([], tf.string),
    "id": tf.io.FixedLenFeature([], tf.string),
  }
  # Converting raw data to a python dictionary with UNLABELLED_TFREC_FORMAT.
  example = tf.io.parse_single_example(example, UNLABELLED_TFREC_FORMAT)
  # Decoding the image.
  image = decode_image(example["image"])
  id_num = example["id"]
  return image, id_num


# This function takes several filenames (tfrecords) and creates a tensorflow
# dataset from them.
def load_dataset(filenames, labelled=True, ordered=False):
  # The order of the images doesn't matter so we are turning on ignore_order
  # this results in faster loading times.
  ignore_order = tf.data.Options()
  if not ordered:
    ignore_order.experimental_deterministic = False

  # Creating the dataset. We are using parallel read option because obviously
  # it results in faster loading times.
  dataset = tf.data.TFRecordDataset(
    filenames, num_parallel_reads=tf.data.experimental.AUTOTUNE
  )
  dataset = dataset.with_options(ignore_order)
  # The final element of the pipeline is taking the raw tfrecord
  # and converting it to labelled on unlabelled input data with the upper functions.
  dataset = dataset.map(
    read_labelled_tfrecord if labelled else read_unlabelled_tfrecord,
    num_parallel_calls=tf.data.experimental.AUTOTUNE,
  )
  return dataset

In [7]:
# This function creates the training dataset.
# repeat will be used, this will make sure that after reaching the end record
# we go back to the first.
# BATCH_SIZE will be used.
# We will prefetch 1 batch for accellerating the reading process.
def get_training_dataset():
  dataset = load_dataset(TRAINING_FILENAMES, labelled=True)
  dataset = dataset.repeat()
  dataset = dataset.batch(BATCH_SIZE)
  dataset = dataset.prefetch(1)
  return dataset

# This function creates the validation dataset.
# BATCH_SIZE will be used.
# We will prefetch 1 batch for accellerating the reading process.
def get_validation_dataset(ordered=False):
  dataset = load_dataset(VALIDATION_FILENAMES, labelled=True, ordered=ordered)
  dataset = dataset.batch(BATCH_SIZE)
  dataset = dataset.prefetch(1)
  return dataset


# This function creates the test dataset.
# BATCH_SIZE will be used.
# We will prefetch 1 batch for accellerating the reading process.
def get_test_dataset(ordered=False):
  dataset = load_dataset(TEST_FILENAMES, labelled=False, ordered=ordered)
  dataset = dataset.batch(BATCH_SIZE)
  dataset = dataset.prefetch(1)
  return dataset


# This function counts the number of tfrecords in a dataset.
def count_data_items(filenames):
  n = [
    int(re.compile(r"-(\d*)\.").search(filename).group(1))
    for filename in filenames
  ]
  return np.sum(n)

In [8]:
# This will be used to calculate the number of batches needed to go through
# all the dataset in an epoch.
num_train_images = count_data_items(TRAINING_FILENAMES)

# Getting the train, validation and test datasets.
train_dataset = get_training_dataset()
validation_dataset = get_validation_dataset()
test_dataset = get_test_dataset()

In [9]:
# For hyperparameter-optimization keras-tuner will be used.
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 3.7 MB/s eta 0:00:00


In [16]:
from tensorflow.keras import datasets, layers, models, callbacks, applications, optimizers
import keras_tuner as kt

def build_model(hp):
  # Using Transfer Learning technique for this project.
  # InceptionV3 pretrained model will be used.
  pre_trained_model = applications.InceptionV3(input_shape=(*IMAGE_SIZE, 3), include_top=False, weights='imagenet')
  # Freezing the weights of the model.
  for layer in pre_trained_model.layers:
    layer.trainable = False

  # Adding our own model on top of InceptionV3.
  x = pre_trained_model.output
  # Using pooling to decrease dimension.
  x = layers.GlobalAveragePooling2D()(x)
  # Putting one hidden layer with units.
  x = layers.Dense(hp.Choice('units', [256, 512]), activation='relu')(x)
  # The output will be a probability distribution of classes.
  x = layers.Dense(NUM_OF_CLASSES, activation='softmax')(x)

  # Creating and compiling the final model from the pretrained model and our own model.
  model = models.Model(pre_trained_model.input, x)
  optimizer = optimizers.Adam(learning_rate=0.001)
  model.compile(
      optimizer=optimizer,
      loss='categorical_crossentropy',
      metrics=['acc']
  )
  return model

# Using keras-tuner for hyperparameter-optimization.
tuner = kt.Hyperband(
    build_model,
    objective='val_acc',
    max_epochs=5,
    factor=3,
    directory='best_model',
    project_name='FlowerClassificationModel',
    overwrite=True
)

# Using EarlyStopping to prevent overfitting.
es = callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='max')

# Searching with Hyperband in the hyperparameter-space.
tuner.search(train_dataset,
    validation_data=validation_dataset,
    epochs=25,
    steps_per_epoch=num_train_images / BATCH_SIZE,
    callbacks=[es])

Trial 2 Complete [00h 01m 31s]
val_acc: 0.6608297228813171

Best val_acc So Far: 0.6737607717514038
Total elapsed time: 00h 04m 02s


In [18]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the optimal hyperparameters and train it on the data for 10 epochs
model = tuner.hypermodel.build(best_hps)

# Saving the best model so far.
cp = callbacks.ModelCheckpoint(filepath='model', monitor='val_acc', mode='max')
# Tensorboard for visualizing.
tb = callbacks.TensorBoard(update_freq=1)

history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=10,
    steps_per_epoch=num_train_images / BATCH_SIZE,
    callbacks=[es, cp, tb])

Epoch 1/10
398/398 [==============================] - 75s 167ms/step - loss: 2.1830 - acc: 0.4848 - val_loss: 1.4900 - val_acc: 0.6166
Epoch 2/10
398/398 [==============================] - 68s 171ms/step - loss: 1.1542 - acc: 0.6904 - val_loss: 1.2703 - val_acc: 0.6692
Epoch 3/10
398/398 [==============================] - 71s 177ms/step - loss: 0.8282 - acc: 0.7717 - val_loss: 1.2263 - val_acc: 0.6840
Epoch 4/10
398/398 [==============================] - 68s 171ms/step - loss: 0.6170 - acc: 0.8331 - val_loss: 1.2471 - val_acc: 0.6786
Epoch 5/10
398/398 [==============================] - 70s 177ms/step - loss: 0.4735 - acc: 0.8730 - val_loss: 1.2914 - val_acc: 0.6794
Epoch 6/10
398/398 [==============================] - 69s 173ms/step - loss: 0.3651 - acc: 0.9039 - val_loss: 1.2879 - val_acc: 0.6880
Epoch 7/10
398/398 [==============================] - 68s 171ms/step - loss: 0.2957 - acc: 0.9214 - val_loss: 1.3378 - val_acc: 0.6913
Epoch 8/10
398/398 [==============================] - 7